# Computer Science Refershments Project 1

## Description

TODO

## Solution: 
_Divide and Conquer !_
Divide the problem when:
- change of segment in the theoretical trajectory
- intersection of the theoretical and experimental trajectories
- reach a _vertex_ in the experimental trajectory

How to calculate the error:
__The sum of the areas of the "Trapezoid" formed by two points on the theoretical trajectory and two points of the experimental trajectory__
o When two points from the experimental and theoretical trajectories are equals, the area is a triangle. 

In [1]:
#imports

import numpy as np
import numpy.linalg as lin

In [2]:
def trajectoryError(coordTh,coordExp):
    """
    arg1 coordTh : a list of tuples (x,y). The path which should be followed.
    arg2 coordExp: a list of tuples (x,y). The points received from the "indoors-gps"
    return: error, the total area difference betweeen the theoretical trajectory and the experimental one divided by the distance.
    """
    ## assume the gps knows the exact start and end of the theoretical path:
    if coordTh[0] != coordExp[0]:
        coordExp = [coordTh[0]] + [coordExp]
    if coordTh[-1] != coordExp[-1]:
        coordExp.append(coordTh[-1])
        
    # initialize values:
    area = 0 # the total area between the theoretical and experimental paths
    distance = path_length(coordTh) # total theoretical path length
    j = 0 # iterator over the experimental path
    i = 0 # iterator over the theoretical path
    
    # start iterations:
    while i+1 <= len(coordTh) and j+1 <= len(coordexp):
        # we work on a subsegment [coordTh[i], coordTh[i+1]] 
        
        # search for an intersection:
        intersectPoint = intersection(coordTh[i], coordTh[i+1], coordexp[j], coordexp[j+1])
        
        # compute the orthogonal projections of the experimental points on the subsegment
        ort1 = ortogonalProjection(coordTh[i], coordTh[i+1], coordexp[j])
        ort2 = ortogonalProjection(coordTh[i], coordTh[i+1], coordexp[j+1]) 
        
        if intersectPoint:
            if intersectPoint == coordTh[i+1] == coordexp[j]:
                # we're outside of the subsegment
                i += 1 # advance along the theoretical path       
            else:
                # Before the intersection:
                # the points form the right triangle -> ort1, intersectPoint, coordexp[j]  
                base = seg_length(coordexp[j], intersectPoint)
                height =  seg_length(coordTh[i], ortogonalProjection(coordexp[j], intersectPoint, coordTh[i]))
                area += areaRightTriangle(base, height)
                
                # After the intersection:
                # the points form the right triangle -> intersectPoint, coordth[i+1], coordexp[i+1]
                base = seg_length(coordexp[j+1], intersectPoint)
                height =  seg_length(coordTh[i+1], ortogonalProjection(coordexp[j+1], intersectPoint, coordTh[i]))
                areaRightTriangle(base, height)
                
                j += 1 # advance along the experimental path  
        else:      
            if ort1 and ort2:
                area += computeArea(ort1, ort2, coordexp[j], coordexp[j+1])
                j += 1 # advance along the experimental path
            elif ort1:
                # area is given by the triangle -> exp[j], ort1, exp[j+1]
                #              and the triangle -> ort1, exp[j+1], th[i+1]
                pass
            elif ort2:
                # area is given by the triangle -> exp[j], th[i], exp[j+1]
                #              and the triangle -> th[i], exp[j+1], ort2
                pass
            else:
                pass
                i += 1 # advance along the theoretical path
                
    return area / distance


def seg_length(x_1,x_2):
    """
    arg1 x_1 : A tuple, the coordinates of the first point of the segment
    arg2 x_2 : A tuple, the coordinates of the second point of the segment
    return : A float, the length of the segment made up by x_1 and x_2
    """
    return(np.sqrt((x_1[0]-x_2[0])**2 + (x_1[1]-x_2[1])**2))


def path_length(path):
    """
    arg1 path : A table of tuples, each tuple representing the coordinates of a point of the path.
    return : leng. A float, 
    """
    assert len(path) > 1
    leng = 0
    for i in range(len(path)-1):
        leng += seg_length(path[i],path[i+1])
    return(leng)


def intersection(x_1, x_2, y_1, y_2):
    """
    arg1,2 x_i: A tuple, the coordinates of a point on the theoretical trajectory
    arg3,4 y_i: A tuple, the coordinates of a point on the experimental trajectory
    return: intersect. A tuple, the coordinates of the intersection or null.
    Remark : if the intersection is not in the two segments, returns None.
    """

    # First case : the two lines are vertical
    if (x_1[0] == x_2[0] and y_1[0]  == y_2[0]) :
        intersect = None
    
    # Second case : only the first line is vertical
    elif x_1[0] == x_2[0] :
        a = (y_1[1]-y_2[1])/(y_1[0]-y_2[0])
        b = y_2[1] - a * y_2[0]
        intersect = (x_1[0],a*x_1[0] + b)
    
    # Third case : only the second line is vertical
    elif y_1[0]  == y_2[0] :
        intersect = intersection(y_1,y_2,x_1,x_2)
    
    # Last case : general case
    else :
        a_x = (x_1[1] - x_2[1])/(x_1[0] - x_2[0])
        b_x = x_1[1] - a_x * x_1[0]
        a_y = (y_1[1] - y_2[1])/(y_1[0] - y_2[0])
        b_y = y_1[1] - a_y * y_1[0]
        
        if a_x == a_y :
            # Case where the lines are parallel
            intersect = None
        
        else :
            #The equations are
            #y = a_x*x + b_x
            #y = a_y*x + b_y
            # The solution is the inverted matrix
            intersect = ( (b_y-b_x)/(a_y-a_x),\
                          (b_y * a_x - b_x * a_y)/(a_y - a_x) )
    
    # If a point is found, check if it belongs to the segments and not only the lines (x_1[0]<intersect[0]<x_2[0] or...)
    if intersect :
        if not ( (intersect[0] >= min(x_1[0],x_2[0])) and (intersect[0] <= max(x_1[0],x_2[0]))\
                and (intersect[1] >= min(x_1[1],x_2[1])) and (intersect[1] <= max(x_1[1],x_2[1]))\
                and (intersect[0] >= min(y_1[0],y_2[0])) and (intersect[0] <= max(y_1[0],y_2[0]))\
                and (intersect[1] >= min(y_1[1],y_2[1])) and (intersect[1] <= max(y_1[1],y_2[1]))): # Checked if it is in the y-segment 
            intersect = None
    
    return intersect


def ortogonalProjection(x_1, x_2, y):
    """
    arg1,2 x_i: A tuple, the coordinates of a point on the theoretical trajectory
    arg3 y: A tuple, the coordinates of a point on the experimental trajectory
    return: newPoint. A tuple, the coordinates of the orthogonal projection on y on the segment [x_1,x_2]
    """
    
    if x_1[0] == x_2[0] :
        # First case : the line containing x_1 and x_2 is vertical
        newPoint = (x_1[0],y[1])
        
    else :
    # Second case : the line containing x_1 and x_2 is not vertical
        a = (x_2[1]-x_1[1])/(x_2[0]-x_1[0])
        b = x_1[1] - a * x_1[0]
        # Now we know the line containing x_1 and x_2 has the equation y = a * x + b
        # So a normal vector to this line is (a,-1)
        # This vector is directing the normal line passing by the point y and its equation is -x + a*y + c = 0 :
        # let's find c!
        c = y[0] + a*y[1]
        # Now the intersection point is the point (x,y) which verifies :
        # a*x - y = -b
        # -x + a*y = -c
        mat = np.array([a,-1],[-1,a])
        # Not invertible if a = +- 1 : ???
        point = np.dot(lin.inv(mat)*,np.array([-b,-c]))
        newPoint = (point[0],point[1])
    
    ## express the line passing by x_1 and x_2
    ## line = (x_2[1]-x_1[1])/(x_2[0]-x_1[0])
    
    return newPoint
                       


def computeArea(x_1, x_2, y_1, y_2): # TODO: retirer cas des triangles.
                       
    """ 
    arg1,2 x_i: A tuple, the coordinates of a point on the theoretical trajectory
    arg3,4 y_i: A tuple, the coordinates of a point on the experimental trajectory
    return: area. An int, the area of the figure formed by the 4 coordinates
    """
    #Cases of triangles
    if x_1 == y_1 :
        # The triangle is formed by the segments x_1,x_2 and x_2,y_2
        area = seg_length(x_1,x_2) * seg_length(x_2,y_2) / 2
    elif x_2 == y_2 :
        area = seg_length(x_1,y_1) * seg_length(x_1,x_2)/2
                              
    # Assert the arguments really form a trapezoid, with (x_1,y_1) being parallel with (x_2,y_2)
    else :
        # Always check they are parallel
        # Case of vertical lines 
        if x_1[0] == y_1[0]:
            assert x_2[0] == y_2[0]
            area = seg_length(x_1,x_2) * 0.5 * seg_length(x_1,y_1) * seg_length(x_2,y_2)
        elif x_2[0] == y_2[0]:
            assert x_1[0] == y_1[0]
            area = seg_length(x_1,x_2) * 0.5 * seg_length(x_1,y_1) * seg_length(x_2,y_2)
            
        else :
            # General case
            assert((y_1[1]-x_1[1])/(y_1[0]-x_1[0]) == (x_2[1]-y_2[1])/(x_2[0]-y_2[0]))
            area = seg_length(x_1,x_2) * 0.5 * seg_length(x_1,y_1) * seg_length(x_2,y_2)
            
    ## Particular case where the xs are the same : vertical lines
    
    
    ## Other case : equality of the slopes
    
    
    # Computing the area
    area = 0
    
    # case 1: only two points -> return 0
    
    # case 2: if two points are equal -> triangle
    
    # case 3: all point distinct -> trapezoid
    
    return area

def areaRightTriangle(base, height):
    """
    Computes the area of a right triangle.
    
    arg1: base. A numeral, the length of the basis of the triangle 
    arg2: height. A numeral, the length of the basis of the triangle
    return: the area of the rectangle
    """
    return np.abs((base * height) / 2)